This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'zFWStsiY37zxSFxaceo7'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

from datetime import datetime

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
test_request = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2018-09-24&end_date=2018-09-25&api_key=zFWStsiY37zxSFxaceo7")

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
t = test_request.json()

#t['dataset']['data'][0]

#t_columns = t['dataset']['column_names']
#t_data = t['dataset']['data']
#print(str(t_columns[0]) + ": " + str(t_data[0][0]) + "\n" + str(t_columns[1]) + ": "+ str(t_data[0][4]))

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# Question 1
# Get ticker data for AFX_X from the Frankfurt Stock Exchange for 2017

r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=zFWStsiY37zxSFxaceo7")

In [6]:
# Question 2
# Convert the JSON data to a Python dictionary

# Note that I'm changing to an ordered dict of named tuples
d = r.json()

# in order to convert the dataset to an ordered dict of named tuples, some prep work:
# create 'columns' and 'data' variables for easier lookup
d_column_names = d['dataset']['column_names']
d_data_rows = d['dataset']['data']

# create a named tuple of daily data, for easier data retrieval
# replace spaces with underscores in the names for the tuple
# starting from element 1 in the column_names since element 0 is the date(will be the key)
names_for_tuple = ' '.join([entry.replace(' ', '_') for entry in d_column_names[1:]])
Daily = collections.namedtuple('Daily', names_for_tuple)

# Convert the whole dataset to an ordered dict of named tuples
# Each day's data is now in the format data['Date']
data = collections.OrderedDict()
for row in d_data_rows:
    data[row[0]] = Daily(*row[1:])
    
#d_data_rows[0][0]
print(data['2017-12-29'])
print(data['2017-12-29'].Traded_Volume)

Daily(Open=51.76, High=51.94, Low=51.45, Close=51.76, Change=None, Traded_Volume=34640.0, Turnover=1792304.0, Last_Price_of_the_Day=None, Daily_Traded_Units=None, Daily_Turnover=None)
34640.0


In [7]:
# Question 3
# Highest and lowest opening prices

def extract(dataset, start_date, end_date, att_name):
    """Extract a particular attribute over a period of time, from the given dataset.
    Input: dataset, start and end dates, and the name of the attribute.
    Output: a list of (attribute, date) tuples.
    """
             
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    return_list = []
    
    try:
        # Loop through dates in date order
        # Convert dates to ordinal (integers) for range
        for ordinal in range(start.toordinal(), end.toordinal()):

            # Convert the integer back to a date for date calculations
            day = datetime.strftime(datetime.fromordinal(ordinal), '%Y-%m-%d')
        
            if day in dataset:
                return_list.append((getattr(dataset[day], att_name), day))
        
        # Pull out any None values before returning
        return_list = [(x, y) for (x, y) in return_list if x is not None]
        return return_list
    
    except:
        print("Try again, checking your parameters:")
        print("Dataset, start date (YYYY-MM-DD), end date, attribute name.")
        
open_data = extract(data, '2017-01-01', '2017-12-31', 'Open')
open_data
open_data.sort()
low, low_date = open_data[0]
high, high_date = open_data[-1]
print('Highest Open: ' + str(round(high,2)) + " on " + high_date)
print('Lowest Open: ' + str(round(low,2)) + " on " + low_date)

Highest Open: 53.11 on 2017-12-14
Lowest Open: 34.0 on 2017-01-24


In [8]:
# Question 4
# Largest change in any one day

highs = extract(data, '2017-01-01', '2017-12-31', 'High')
lows = extract(data, '2017-01-01', '2017-12-31', 'Low')
b = list(zip(highs,lows))

# Create a list of tuples: daily difference high - low and date, where the dates are the same
diffs = [(t[0][0] - t[1][0], t[0][1]) for t in b if t[0][1] == t[1][1]]

# Check that the lengths of the 3 lists are the same
# Since we're comparing on date equality, this means that each list has exactly the same dates
# print(str(len(highs)) + " " + str(len(lows)) + " " + str(len(diffs)))

diffs.sort()
big_change, day = diffs[-1]
print("The largest change in any one day was " + str(round(big_change,2)) + " on " + day + ".")

The largest change in any one day was 2.81 on 2017-05-11.


In [9]:
# Question 5
# Largest change between any two days

closes = extract(data, '2017-01-01', '2017-12-31', 'Close')
z = list(zip(closes, closes[1:]))
# Create a tuple of the change and the ending date
changes = [(t[1][0] - t[0][0], t[1][1]) for t in z]
changes.sort()
big_neg, neg_date = changes[0]
big_pos, pos_date = changes[-1]
print('The largest negative change was ' + str(round(big_neg, 2)) + ' on ' + neg_date + '.')
print('The largest positive change was ' + str(round(big_pos, 2)) + ' on ' + pos_date + '.')

The largest negative change was -2.56 on 2017-08-09.
The largest positive change was 1.72 on 2017-05-11.


In [10]:
# Question 6
# Average daily trading volume

def average(list):
    """I could swear there's a list average in the standard library, but I didn't see one, so here's my own."""
    
    if list == []:
        return None
    else:
        return sum(list) / len(list)
    
volumes = extract(data, '2017-01-01', '2017-12-31', 'Traded_Volume')
avg_volume = average([tup[0] for tup in volumes])
print('The average volume for the year was: ' + "{:,}".format(round(avg_volume,2)) + '.')

The average volume for the year was: 89,124.34.


In [11]:
# Question 7
# Median trading volume

def median(list):
    """Median of a list, where the median for an even-length list is the average of the two median values.
    Input: a list.
    Output: None for an empty list, or the median (for an even-length list, the average of the two median values).
    """
    
    if list == []:
        return None
    else:
        length = len(list)
        if length % 2 == 0:
            return (list[length // 2 - 1] + list[length // 2]) / 2
        else:
            return list[length // 2]

        
volumes = extract(data, '2017-01-01', '2017-12-31', 'Traded_Volume')
median_volume = median([tup[0] for tup in volumes])
print('The median volume for the year was: ' + "{:,}".format(round(median_volume,2)) + '.')

The median volume for the year was: 134,965.0.
